In [ ]:
# Install jieba tool
!pip install jieba --user
!pip install gensim --user
!pip install wordcloud --user

## Introduction

Word2Vec is a popular algorithm used for generating dense vector representations of words in large corpora using unsupervised learning. The resulting vectors have been shown to capture semantic relationships between the corresponding words and are used extensively for many downstream natural language processing (NLP) tasks like sentiment analysis, named entity recognition and machine translation.  

SageMaker BlazingText which provides efficient implementations of Word2Vec on

- single CPU instance
- single instance with multiple GPUs - P2 or P3 instances
- multiple CPU instances (Distributed training)

In this notebook, we demonstrate how BlazingText can be used for distributed training of word2vec using multiple CPU instances.

## Setup

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. 
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket() # Replace with your own bucket name if needed
prefix = 'gcr_sagemaker_workshop/NLP/word2vec' #Replace with the prefix under which you want to store the data if needed
region_name = boto3.Session().region_name

In [ ]:
!mkdir -p data

### Wiki Chinese raw corpus download

Next, we download a chinese dataset of wiki Chinese, it is pre-processed from a xml compess package, and is tranformed to simple chinese. Total size 1.3G

In [ ]:
!wget https://421710401846-sagemaker-us-west-2.s3-us-west-2.amazonaws.com/nlp-handson/wiki_zh_sample data/wiki_zh_sample
    

### cleaning the raw data 
- The raw data contains a lot English words,Arabic numerals and punctuation marks, remove these noising information and then use Jieba to parse chinese words.
- While parsing words, we will encounter many meaningless words like "的、虽然、因为", we call these words stop words, also need to remove

In [ ]:
# Download stopwords
!wget https://421710401846-sagemaker-us-west-2.s3-us-west-2.amazonaws.com/nlp-handson/zhstopwords.txt data/zhstopwords.txt
    

In [ ]:
import logging,jieba,os,re

In [ ]:
def get_stopwords():
    logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',level=logging.INFO)
    #加载停用词表
    stopword_set = set()
    with open("data/zhstopwords.txt",'r',encoding="utf-8") as stopwords:
        for stopword in stopwords:
            stopword_set.add(stopword.strip("\n"))
    return stopword_set

In [ ]:
def parse_zh_words(read_file_path,save_file_path): 
    file = open(read_file_path,"r",encoding="utf-8")
    #过滤掉英文和数字等特殊字符
    r1 = '[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    #写文件
    output = open(save_file_path,"w+",encoding="utf-8")
    content_line = file.readline()
    #获取停用词表
    stopwords = get_stopwords()
    #定义一个字符串变量，表示一篇文章的分词结果
    article_contents = ""
    while content_line:
        content_line = content_line.strip("\n")
        if len(content_line) > 0:
            #去除数字和英文等特殊字符
            zh_content_line = re.sub(r1, '', content_line)
            #使用jieba进行分词
            words = jieba.cut(zh_content_line,cut_all=False)
            for word in words:
                if word not in stopwords:
                    article_contents += word+" "
            if len(article_contents) > 0:
                output.write(article_contents+"\n")
                article_contents = ""
        content_line = file.readline()
    output.close()

Start parsing chinese words

In [ ]:
input_file = './data/wiki_zh_sample'
output_file = './data/wiki_zh_corpus'
parse_zh_words(input_file, output_file)

Let us take a look at the first 10 lines of the corpus.

In [ ]:
f = open("./data/wiki_zh_corpus", 'r', encoding='utf-8')
if(f):
    for i in range(10):
        print(f.readline(), end='')

After the corpus data's processing is completed, we need to upload it to S3 so that it can be consumed by SageMaker to execute training jobs. 

In [ ]:
train_channel = prefix + '/zh-train'

sess.upload_data(path='data/wiki_zh_corpus', bucket=bucket, key_prefix=train_channel)

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)

Next we need to setup an output location at S3, where the model artifact will be dumped. These artifacts are also the output of the algorithm's training job.

In [ ]:
s3_output_location = 's3://{}/{}/zh-output'.format(bucket, prefix)

## Training Setup
Now that we are done with all the setup that is needed, we are ready to train our object detector. To begin, let us create a ``sageMaker.estimator.Estimator`` object. This estimator will launch the training job.

In [ ]:
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

## Training the BlazingText model for generating word vectors

Similar to the original implementation of [Word2Vec](https://arxiv.org/pdf/1301.3781.pdf), SageMaker BlazingText provides an efficient implementation of the continuous bag-of-words (CBOW) and skip-gram architectures using Negative Sampling, on CPUs and additionally on GPU[s]. The GPU implementation uses highly optimized CUDA kernels. To learn more, please refer to [*BlazingText: Scaling and Accelerating Word2Vec using Multiple GPUs*](https://dl.acm.org/citation.cfm?doid=3146347.3146354). BlazingText also supports learning of subword embeddings with CBOW and skip-gram modes. This enables BlazingText to generate vectors for out-of-vocabulary (OOV) words, as demonstrated in this [notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/blazingtext_word2vec_subwords_text8/blazingtext_word2vec_subwords_text8.ipynb).




Besides skip-gram and CBOW, SageMaker BlazingText also supports the "Batch Skipgram" mode, which uses efficient mini-batching and matrix-matrix operations ([BLAS Level 3 routines](https://software.intel.com/en-us/mkl-developer-reference-fortran-blas-level-3-routines)). This mode enables distributed word2vec training across multiple CPU nodes, allowing almost linear scale up of word2vec computation to process hundreds of millions of words per second. Please refer to [*Parallelizing Word2Vec in Shared and Distributed Memory*](https://arxiv.org/pdf/1604.04661.pdf) to learn more.

BlazingText also supports a *supervised* mode for text classification. It extends the FastText text classifier to leverage GPU acceleration using custom CUDA kernels. The model can be trained on more than a billion words in a couple of minutes using a multi-core CPU or a GPU, while achieving performance on par with the state-of-the-art deep learning text classification algorithms. For more information, please refer to [algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html) or [the text classification notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/blazingtext_text_classification_dbpedia/blazingtext_text_classification_dbpedia.ipynb).

To summarize, the following modes are supported by BlazingText on different types instances:

|          Modes         	| cbow (supports subwords training) 	| skipgram (supports subwords training) 	| batch_skipgram 	| supervised |
|:----------------------:	|:----:	|:--------:	|:--------------:	| :--------------:	|
|   Single CPU instance  	|   ✔  	|     ✔    	|        ✔       	|  ✔  |
|   Single GPU instance  	|   ✔  	|     ✔    	|                	|  ✔ (Instance with 1 GPU only)  |
| Multiple CPU instances 	|      	|          	|        ✔       	|     | |

Now, let's define the resource configuration and hyperparameters to train word vectors on *text8* dataset, using "batch_skipgram" mode on two c4.2xlarge instances.


In [ ]:
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         #train_instance_type='ml.c4.4xlarge',
                                         train_instance_type='ml.p3.2xlarge',
                                         train_volume_size = 10,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

Please refer to [algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html) for the complete list of hyperparameters.

In [ ]:
bt_model.set_hyperparameters(mode= "skipgram", #"batch_skipgram",
                             epochs=10,
                             min_count=5,
                             sampling_threshold=0.0001,
                             learning_rate=0.05,
                             window_size=5,
                             vector_dim=100,
                             negative_samples=5,
                             batch_size=11, #  = (2*window_size + 1) (Preferred. Used only if mode is batch_skipgram)
                             evaluation=False,#  Do not Perform similarity evaluation on WS-353 dataset at the end of training
                             subwords=False) # Subword embedding learning is not supported by batch_skipgram

Now that the hyper-parameters are setup, let us prepare the handshake between our data channels and the algorithm. To do this, we need to create the `sagemaker.session.s3_input` objects from our data channels. These objects are then put in a simple dictionary, which the algorithm consumes.

In [ ]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data}

We have our `Estimator` object, we have set the hyper-parameters for this object and we have our data channels linked with the algorithm. The only  remaining thing to do is to train the algorithm. The following command will train the algorithm. Training the algorithm involves a few steps. Firstly, the instance that we requested while creating the `Estimator` classes is provisioned and is setup with the appropriate libraries. Then, the data from our channels are downloaded into the instance. Once this is done, the training job begins. The provisioning and data downloading will take some time, depending on the size of the data. Therefore it might be a few minutes before we start getting training logs for our training jobs. The data logs will also print out `Spearman's Rho` on some pre-selected validation datasets after the training job has executed. This metric is a proxy for the quality of the algorithm. 

Once the job has finished a "Job complete" message will be printed. The trained model can be found in the S3 bucket that was setup as `output_path` in the estimator.

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

## Hosting / Inference
Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same type of instance that we used to train. Because instance endpoints will be up and running for long, it's advisable to choose a cheaper instance for inference.

In [ ]:
bt_endpoint = bt_model.deploy(initial_instance_count = 1,instance_type = 'ml.m4.xlarge')

### Getting vector representations for words

#### Use JSON format for inference
The payload should contain a list of words with the key as "**instances**". BlazingText supports content-type `application/json`.

In [ ]:
# words = ["awesome", "blazing"]
words = ["故宫", "紫禁城"]

payload = {"instances" : words}

response = bt_endpoint.predict(json.dumps(payload))

vecs = json.loads(response)
print(vecs)

As expected, we get an n-dimensional vector (where n is vector_dim as specified in hyperparameters) for each of the words. If the word is not there in the training dataset, the model will return a vector of zeros.

### Evaluation

Let us now download the word vectors learned by our model and visualize them using a [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) plot.

In [ ]:
s3 = boto3.resource('s3')

key = bt_model.model_data[bt_model.model_data.find("/", 5)+1:]
print(key)
s3.Bucket(bucket).download_file(key, 'zh-model.tar.gz')

Uncompress `model.tar.gz` to get `vectors.txt`

In [ ]:
!tar -xvzf zh-model.tar.gz

If you set "evaluation" as "true" in the hyperparameters, then "eval.json" will be there in the model artifacts.

The quality of trained model is evaluated on word similarity task. We use [WS-353](http://alfonseca.org/eng/research/wordsim353.html), which is one of the most popular test datasets used for this purpose. It contains word pairs together with human-assigned similarity judgments.

The word representations are evaluated by ranking the pairs according to their cosine similarities, and measuring the Spearmans rank correlation coefficient with the human judgments.

Let's look at the evaluation scores which are there in eval.json. For embeddings trained on the text8 dataset, scores above 0.65 are pretty good.

Now, let us do a 2D visualization of the word vectors

In [ ]:
import numpy as np
from sklearn.preprocessing import normalize

# Read the 400 most frequent word vectors. The vectors in the file are in descending order of frequency.
num_points = 400

first_line = True
index_to_word = []
with open("vectors.txt","r") as f:
    for line_num, line in enumerate(f):
        if first_line:
            dim = int(line.strip().split()[1])
            word_vecs = np.zeros((num_points, dim), dtype=float)
            first_line = False
            continue
        line = line.strip()
        word = line.split()[0]
        vec = word_vecs[line_num-1]
        for index, vec_val in enumerate(line.split()[1:]):
            vec[index] = float(vec_val)
        index_to_word.append(word)
        if line_num >= num_points:
            break
word_vecs = normalize(word_vecs, copy=False, return_norm=False)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(perplexity=40, n_components=2, init='pca', n_iter=10000)
two_d_embeddings = tsne.fit_transform(word_vecs[:num_points])
labels = index_to_word[:num_points]

### Becasue we need matplot print chinese words, we need to install SimHei font into notebook instance.
https://blog.csdn.net/wlwlwlwl015/article/details/51482065 The simhei font can be download from https://421710401846-sagemaker-us-west-2.s3-us-west-2.amazonaws.com/nlp-handson/simhei.ttf

In [ ]:
'''
from matplotlib.font_manager import _rebuild
_rebuild()
'''

In [ ]:
from matplotlib import pylab
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
%matplotlib inline

def plot(embeddings, labels):
    pylab.figure(figsize=(20,20))
    for i, label in enumerate(labels):
        x, y = embeddings[i,:]
        pylab.scatter(x, y)
        pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                       ha='right', va='bottom')
    #pylab.savefig('tsne-ch.png')
    pylab.show()

plot(two_d_embeddings, labels)

Running the code above might generate a plot like the one below. t-SNE and Word2Vec are stochastic, so although when you run the code the plot won’t look exactly like this, you can still see clusters of similar words such as below where 'british', 'american', 'french', 'english' are near the bottom-left, and 'military', 'army' and 'forces' are all together near the bottom.

In [ ]:
from gensim.models import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('vectors.txt', binary=False)

similarity = word_vectors.similarity('故宫', '紫禁城')
print("{:.4f}".format(similarity))

result1 = word_vectors.most_similar(positive=['女人', '国王'], negative=['男人'])
print("{}: {:.4f}".format(*result1[0]))

print(word_vectors.doesnt_match("天才 傻瓜 疯子 牛肉".split()))

result2 = word_vectors.similar_by_word("太湖",topn=5)
for i in range(5):
    print("{}: {:.4f}".format(*result2[i]))

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

'''
获取一个圆形的mask
'''
def get_mask():
    x,y = np.ogrid[:500,:500]
    mask = (x-250) ** 2 + (y-250)**2 > 250 ** 2
    mask = 255 * mask.astype(int)
    return mask

'''
绘制词云
'''
def draw_word_cloud(word_cloud):
    wc = WordCloud(font_path='./simhei.ttf', background_color="white",mask=get_mask())
    wc.generate_from_frequencies(word_cloud)
    #隐藏x轴和y轴
    plt.axis("off")
    plt.imshow(wc,interpolation="bilinear")
    plt.show()

#输入一个词找出相似的前100个词
one_corpus = ["人工智能"]
result = word_vectors.most_similar(one_corpus[0],topn=100)
#将返回的结果转换为字典,便于绘制词云
word_cloud = dict()
for sim in result:
    word_cloud[sim[0]] = sim[1]
#绘制词云
draw_word_cloud(word_cloud)


### Stop / Close the Endpoint (Optional)
Finally, we should delete the endpoint before we close the notebook.

In [ ]:
sess.delete_endpoint(bt_endpoint.endpoint)